# Pytorch -- Feed Forward Neural Network

#### Importing required libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

#### Device configuration

In [2]:
device = torch.device('cpu')

In [3]:
device

device(type='cpu')

#### Hyper-parameters 

In [4]:
input_size = 784
hidden_size = 128
num_classes = 10
num_epochs = 3
batch_size = 32
learning_rate = 0.001

#### Dataset --  MNIST

In [5]:
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

#### Data loader

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Fully connected neural network with one hidden layer

In [7]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [8]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

#### Loss and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

#### Train the model

In [10]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))



Epoch [1/3], Step [100/1875], Loss: 0.4636
Epoch [1/3], Step [200/1875], Loss: 0.5625
Epoch [1/3], Step [300/1875], Loss: 0.3399
Epoch [1/3], Step [400/1875], Loss: 0.2878
Epoch [1/3], Step [500/1875], Loss: 0.5121
Epoch [1/3], Step [600/1875], Loss: 0.2681
Epoch [1/3], Step [700/1875], Loss: 0.3077
Epoch [1/3], Step [800/1875], Loss: 0.1548
Epoch [1/3], Step [900/1875], Loss: 0.0884
Epoch [1/3], Step [1000/1875], Loss: 0.1818
Epoch [1/3], Step [1100/1875], Loss: 0.1928
Epoch [1/3], Step [1200/1875], Loss: 0.0952
Epoch [1/3], Step [1300/1875], Loss: 0.2782
Epoch [1/3], Step [1400/1875], Loss: 0.1804
Epoch [1/3], Step [1500/1875], Loss: 0.2135
Epoch [1/3], Step [1600/1875], Loss: 0.3319
Epoch [1/3], Step [1700/1875], Loss: 0.1809
Epoch [1/3], Step [1800/1875], Loss: 0.0859
Epoch [2/3], Step [100/1875], Loss: 0.2102
Epoch [2/3], Step [200/1875], Loss: 0.1055
Epoch [2/3], Step [300/1875], Loss: 0.0743
Epoch [2/3], Step [400/1875], Loss: 0.2077
Epoch [2/3], Step [500/1875], Loss: 0.0477
Ep

### Test the model
    In this phase, we don't need to compute gradients (helpful for memory efficiency)

In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 97.2 %


### Save the model checkpoint

In [ ]:
torch.save(model.state_dict(), 'model.ckpt')